In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
#extracting the robots.txt

url = 'http://www.cars.com/robots.txt'
response  = requests.get(url)
print(response.text)

User-agent: *

Disallow: /windshield/
Disallow: /ajax/coreshoppingsupportapi/1.0/listing/vhr/* # CarFax
Disallow: /ajax/listingsapi/1.0/saved-vehicles/* #Missing Registration cookie
Disallow: /profile/ # User profiles
Disallow: /document-generator/ # Service Coupons - no longer offered

Disallow: /dv/
Disallow: /rv/

Disallow: /react-shop-webapp/

Disallow: /go/advice/advice_carprices.jhtml
Disallow: /go/buy-cars/*
Disallow: /go/car-dealers/kcj/*
Disallow: /go/dealersearch/logDealerLead*
Disallow: /go/search/newBuyIndex.jsp*
Disallow: /go/search/dealerWebsite.jsp*
Disallow: /go/video/motorweek/videoPopup.jsp*

Disallow: /shopping/certified-preowned/
Disallow: /shopping/new/
Disallow: /shopping/results/
Disallow: /dealers/*/inventory/
Disallow: *ni=1*
Disallow: *ni=2*
Disallow: *ni=3*

Disallow: */specialoffers/

Disallow: /vehicle/
Disallow: /vehicledetail/

#Individual Review Crawl Control
Disallow: /dealers/*/reviews/*/
Disallow: /research/*/consumer-reviews/*/
Disallow: */write-revi

In [3]:
#creating a fake user-agent
from fake_useragent import UserAgent

ua = UserAgent()
user_agent = {'User-agent': ua.random}
print(user_agent)

{'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; de-DE) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.3 Safari/533.19.4'}


In [4]:
#creating a header based on the user-agent
header = {
    "User-Agent": 'Mozilla/5.0 (Windows; U; Windows NT 6.1; de-DE) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.3 Safari/533.19.4',
    "Accept-Language": 'en-US'
}


#defining a function to extraxt one page
def get_unit_page_urls(url):
    """
    returns the list of all the items in the input url.
    
    """

    response = requests.get(url, headers=header)
    data = response.text
    soup = bs(data, "html.parser")
    unit_page_urls = []
    
    class_name = "vehicle-card-main js-gallery-click-card"

    for i in range(len(soup.find_all(class_=class_name))):
        url = soup.find_all(class_=class_name)[i].find('a')['href']
        unit_page_urls.append(f"https://www.cars.com/{url}")

    return unit_page_urls

In [5]:
#defining a function to extraxt all pages based on zipcode
def get_multi_urls(start_page, num_of_pages, radius, zipcode):
    """
    returns the list of urls based on location(zipcode).
    
    """

    multi_page_urls = []

    for i in range(num_of_pages):
        root_url = "https://www.cars.com/for-sale/searchresults.action/?page={}&perPage=100&rd={}&searchSource=GN_BREADCRUMB&sort=relevance&zc={}"\
        .format(i + start_page, radius, zipcode)
        
        multi_page_urls.append(root_url)

#         multi_page_urls.append(get_unit_page_urls(root_url)) #appending all urls using get_unit_page_urls function

#     urls_list = [item for sub_list in multi_page_urls for item in sub_list]

    return multi_page_urls

In [6]:
def get_car_features(url):
    cars_list = []
    response = requests.get(url, headers=header)
    data = response.text
    soup = bs(data, "html.parser")
    
    class_name="fancy-description-list"
    main_features = soup.find_all(class_=class_name) #access all the main features
    
    main_feature_list = []
    for i in range(len(main_features)):
        main_feature_list.append(main_features[i].text.strip().replace("\n \n\n", "\n").replace("\n\n\n\n\n\n\n\n\n","\n").replace("\n\n\n\n", "\n").replace("\n\n", "\n").split("\n"))

    basic_features = main_feature_list[0]
    
    extra_features = main_feature_list[1]
    
    basic_dict = {}
    
    features = ['Name', 'Address', 'Rating', 'Mileage', 'Fuel Type', 'City MPG','Highway MPG', 'Engine', 'Drivetrain',
              'Exterior Color', 'Interior Color','Transmission', 'Safety', 'Entertainment', 'Price', 'MPG']
    features_lower = [x.lower() for x in features]

    for i in range(0, len(basic_features)):
        if basic_features[i].lower() in features_lower:
            basic_dict[basic_features[i]] = basic_features[i+1]
        
    extra_features = [x for x in extra_features if x != '']
    for i in range(0, len(extra_features)):
        if extra_features[i].lower() in features_lower:
            basic_dict[extra_features[i]] = extra_features[i+1]
        
    name = soup.find('h1', class_='listing-title').text
    basic_dict['name'] = name
    
    address = soup.find(class_='dealer-address').text.strip()
    basic_dict['address'] = address

    rating = soup.find(class_='sds-rating__count').text
    basic_dict['rating'] = rating

    price = soup.find(class_='primary-price').text
    basic_dict['price'] = price

    MPGs = basic_dict['MPG'].split("–")
    basic_dict['city_MPG'] = MPGs[0]
    basic_dict['highway_MPG'] = MPGs[1]
    
    return basic_dict

In [7]:
import pandas as pd
import time, os
import random



list_of_url = get_multi_urls(start_page=1, num_of_pages=10, radius=20, zipcode=98125)
print(list_of_url)
# cars_list = []


# for url in list_of_url:
#     try:
#         cars_list.append(get_car_features(url))
#     except:
#         print('---')

# df = pd.DataFrame(cars_list)  
# df.head()

['https://www.cars.com/for-sale/searchresults.action/?page=1&perPage=100&rd=20&searchSource=GN_BREADCRUMB&sort=relevance&zc=98125', 'https://www.cars.com/for-sale/searchresults.action/?page=2&perPage=100&rd=20&searchSource=GN_BREADCRUMB&sort=relevance&zc=98125', 'https://www.cars.com/for-sale/searchresults.action/?page=3&perPage=100&rd=20&searchSource=GN_BREADCRUMB&sort=relevance&zc=98125', 'https://www.cars.com/for-sale/searchresults.action/?page=4&perPage=100&rd=20&searchSource=GN_BREADCRUMB&sort=relevance&zc=98125', 'https://www.cars.com/for-sale/searchresults.action/?page=5&perPage=100&rd=20&searchSource=GN_BREADCRUMB&sort=relevance&zc=98125', 'https://www.cars.com/for-sale/searchresults.action/?page=6&perPage=100&rd=20&searchSource=GN_BREADCRUMB&sort=relevance&zc=98125', 'https://www.cars.com/for-sale/searchresults.action/?page=7&perPage=100&rd=20&searchSource=GN_BREADCRUMB&sort=relevance&zc=98125', 'https://www.cars.com/for-sale/searchresults.action/?page=8&perPage=100&rd=20&sear

In [ ]:
cars_list = []
for url in list_of_url:
    try:
        cars_list.append(get_car_features(url))
    except:
        print('---')

In [8]:
# import pandas as pd
# import time, os
# import random

# seattle_zipcode = 98125
# start_page = 1

# for i in range(5): 
#     list_of_url = get_multi_urls(start_page=start_page, num_of_pages=10, radius=30, zipcode=seattle_zipcode)
    
#     cars_list = []

#     for url in list_of_url:
#         try:
#             cars_list.append(get_car_features(url))
#         except:
#             print('---')

#     df = pd.DataFrame(cars_list)
    
# df.head()

In [9]:
#df.to_csv(r'C:\Users\nesha\Desktop\cars_data\df_1000_{}_p{}_p{}.csv'.format(seattle_zipcode, start_page, start_page+9))
    
    #print("Done for page {} to page {}".format(start_page, start_page+9))
    
    #start_page += 10
    
    #time.sleep(15)

In [ ]:

portland_zipcode = 97035
start_page = 1

for i in range(5): 
    list_of_url = get_multi_urls(start_page=start_page, num_of_pages=10, radius=30, zipcode=portland_zipcode)
    
    cars_list = []

    for url in list_of_url:
        try:
            cars_list.append(get_car_features(url))
        except:
            print('---')

    df = pd.DataFrame(cars_list)
    
df.head()

In [ ]:
df.to_csv(r'C:\Users\nesha\Desktop\cars_data\df_1000_{}_p{}_p{}.csv'.format(portland_zipcode, start_page, start_page+9))
    
    print("Done for page {} to page {}".format(start_page, start_page+9))
    
    start_page += 10
    
    time.sleep(15)